In [ ]:
import os
import sys
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
# os.environ['CUDA_HOME'] = '/u/local/cuda/12.3'
# os.environ['PATH'] = f"/u/local/cuda/12.3/bin:{os.environ['PATH']}"
# from IPython.display import display, HTML
# adjust width % as desired
# display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
# clean unused memory
# import gc
# import psutil
# gc.collect()
# psutil.virtual_memory()

In [ ]:
# !pip --version
# !conda --version
# !nvcc --version
# !mamba --version

In [ ]:
# !conda env list
# !conda list
# !mamba env list
# !mamba list
# !pip list

In [ ]:
# !conda create -y -n my-conda-env
# !source activate my-conda-env
# !conda create -n rapids-24.10 -c rapidsai-nightly -c conda-forge -c nvidia  \
#     rapids=24.10 python=3.9 'cuda-version>=12.0,<=12.5' -y

In [ ]:
# Install cuDF and dask_cudf from the RAPIDS channel
# !mamba create --prefix ~/.conda/envs/my_rapids_env -c rapidsai-nightly -c conda-forge -c nvidia  \
#     rapids=24.10 python=3.10 'cuda-version>=12.0,<=12.5' -y

In [ ]:
# !pip install pygam -U
# !pip install aiofiles
# !pip install methylsuite
# !pip install mepylome
# !pip install \
#     --extra-index-url=https://pypi.anaconda.org/rapidsai-wheels-nightly/simple \
#     cudf-cu12 dask-cudf-cu12 dask-cuda --only-binary=:all:
# !pip install pandas==2.2.1 FuzzyTM numpy pyaging scipy seaborn matplotlib -U
#!pip install numpy pyaging --no-cache-dir -U
# !pip cache purge
# run in terminal: conda clean --all -y
# !pip install notebook pyqt5 pyqtwebengine -U

In [ ]:
import inspect
import shutil
import json
import cupy as cp
#import cudf
#import dask_cudf
#from dask_cuda import LocalCUDACluster
#import dask
#from dask.distributed import Client, wait
import pyarrow.feather as fth
#import dask.dataframe as dd
#import torch
import pickle
import pandas as pd
import pyaging as pya
from pygam import LinearGAM, LogisticGAM, s
import numpy as np
import scipy.sparse as sp
from scipy import stats, sparse
import seaborn as sns
import matplotlib.pyplot as plt
from numba import jit, prange
import matplotlib.gridspec as gridspec
from mepylome import Manifest
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.gam.api import GLMGam, BSplines
from statsmodels.stats.anova import anova_lm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer  # Enable experimental features first
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from patsy import dmatrix
#import tensorflow as tf
#tf.get_logger().setLevel('ERROR')
#from tensorflow.keras.layers import Input, Dense
#from tensorflow.keras.models import Model
from scipy.stats import pearsonr

In [ ]:
# !pip freeze > requirements.txt

In [ ]:
# Define paths for Feather files
subset_path_feather = '/u/scratch/c/cobeaman/mymetharray_subset_2458_r_654_c_08152024_192025.feather'
final_path_feather = '/u/scratch/c/cobeaman/mymetharray_final_2458_r_731791_c_08152024_192025.feather'

# Load the data using the single-threaded approach
methylation_data_subset = fth.read_table(subset_path_feather)
methylation_data_final = fth.read_table(final_path_feather)

# Convert to pandas DataFrame and set 'SampleID' as index
methylation_data_subset_pd = methylation_data_subset.to_pandas().set_index('SampleID')
methylation_data_final_pd = methylation_data_final.to_pandas().set_index('SampleID')

In [ ]:
# Check data types
# print(methylation_data_subset['Female'].dtype)

# Ensure 'Female' column is binary (0 or 1)
# methylation_data_final['Female'] = (methylation_data_final['Female'] == 1).astype(int)
# methylation_data_subset['Female'] = (methylation_data_subset['Female'] == 1).astype(int)

# Handle any missing data (if necessary)
methylation_data_final_pd.dropna(inplace=True)
methylation_data_subset_pd.dropna(inplace=True)

# Check for missing values
print(methylation_data_final_pd.isnull().sum())
print(methylation_data_subset_pd.isnull().sum())

# Aggregate any duplicated probes
# methylation_data_subset_pd = pya.pp.epicv2_probe_aggregation(methylation_data_subset_pd)
# methylation_data_final_pd = pya.pp.epicv2_probe_aggregation(methylation_data_final_pd)

In [ ]:
@jit(nopython=True, parallel=True)
def parallel_to_numeric(arr):
    result = np.empty(arr.shape, dtype=np.float64)
    for i in prange(arr.shape[0]):
        for j in prange(arr.shape[1]):
            try:
                result[i, j] = float(arr[i, j])
            except ValueError:
                result[i, j] = np.nan
    return result

def prepare_data_for_adata(df, metadata_cols, chunk_size=1000):
    # Separate metadata and data
    metadata = df[metadata_cols]
    data = df.drop(columns=metadata_cols)
    
    # First, try to convert all columns to numeric
    data = data.apply(pd.to_numeric, errors='coerce')
    
    # Process data in chunks
    chunks = []
    for i in range(0, len(data), chunk_size):
        chunk = data.iloc[i:i+chunk_size]
        
        # Convert to GPU array
        chunk_gpu = cp.array(chunk.values, dtype=cp.float64)
        
        # Transfer back to CPU
        chunk_np = cp.asnumpy(chunk_gpu)
        
        chunks.append(pd.DataFrame(chunk_np, columns=chunk.columns, index=chunk.index))
    
    # Combine processed chunks
    data_processed = pd.concat(chunks)
    
    # Combine metadata and converted data
    df_prepared = pd.concat([metadata, data_processed], axis=1)
    
    return df_prepared

# Prepare the data
methylation_data_final_pd_prepared = prepare_data_for_adata(methylation_data_final_pd, ['Female', 'Age', 'Diagnosis'])
methylation_data_subset_pd_prepared = prepare_data_for_adata(methylation_data_subset_pd, ['Female', 'Age', 'Diagnosis'])

In [ ]:
for df in [methylation_data_final_pd_prepared, methylation_data_subset_pd_prepared]:
    df['female'] = df['Female']
    df['age'] = df['Age']

In [ ]:
import pandas as pd

# Function to get probe data from manifests
def get_probes_from_manifest(manifest_name):
    manifest = Manifest(manifest_name)
    return set(manifest.data_frame['IlmnID'])

# Extract probe IDs from each array as sets for faster lookups
epicv2_probes_set = get_probes_from_manifest("epicv2")
epic_probes_set = get_probes_from_manifest("epic")
i450_probes_set = get_probes_from_manifest("450k")

# Load GrimAge2 feature sets and extract unique probes as a set
df_grimage_subcomponents = pd.read_csv('grimage2_subcomponents.csv', index_col=0)
grimage2_probes_set = set([probe for probe in df_grimage_subcomponents['var'].unique() if probe.startswith('cg')])

# Filter probe names that start with 'cg' from the methylation data
cg_probes_set = set([col for col in methylation_data_final_pd_prepared.columns if col.startswith('cg')])

# Check for missing probes in each array using set difference for efficiency
missing_probes_epicv2 = grimage2_probes_set - epicv2_probes_set
missing_probes_epic = grimage2_probes_set - epic_probes_set
missing_probes_i450 = grimage2_probes_set - i450_probes_set

missing_probes_epicv2_BP = cg_probes_set - epicv2_probes_set
missing_probes_epic_BP = cg_probes_set - epic_probes_set
missing_probes_i450_BP = cg_probes_set - i450_probes_set

# Organize the summary data
summary_data = {
    'Array': ['EPICv2', 'EPIC', '450k', 'GrimAge2'],
    'Total Probes': [
        len(epicv2_probes_set),
        len(epic_probes_set),
        len(i450_probes_set),
        len(grimage2_probes_set)
    ],
    'First Few Probes': [
        ', '.join(list(epicv2_probes_set)[:5]),
        ', '.join(list(epic_probes_set)[:5]),
        ', '.join(list(i450_probes_set)[:5]),
        ', '.join(list(grimage2_probes_set)[:5])
    ],
    'Missing Probes in GrimAge2': [
        len(missing_probes_epicv2),
        len(missing_probes_epic),
        len(missing_probes_i450),
        None
    ],
    'First Few Missing in GrimAge2': [
        ', '.join(list(missing_probes_epicv2)[:5]),
        ', '.join(list(missing_probes_epic)[:5]),
        ', '.join(list(missing_probes_i450)[:5]),
        None
    ],
    'Missing Probes in BPDNAm': [
        len(missing_probes_epicv2_BP),
        len(missing_probes_epic_BP),
        len(missing_probes_i450_BP),
        None
    ],
    'First Few Missing in BPDNAm': [
        ', '.join(list(missing_probes_epicv2_BP)[:5]),
        ', '.join(list(missing_probes_epic_BP)[:5]),
        ', '.join(list(missing_probes_i450_BP)[:5]),
        None
    ],
    'Number of Missing GrimAge2 Probes in Methylation Data': [
        None,
        None,
        None,
        len(grimage2_probes_set - cg_probes_set)
    ],
    'First Few Missing GrimAge2 Probes in Methylation Data': [
        None,
        None,
        None,
        ', '.join(list(grimage2_probes_set - cg_probes_set)[:5])
    ]
}

# Convert to DataFrame for exporting
summary_df = pd.DataFrame(summary_data)

# Save to CSV for sharing
summary_df.to_csv('probe_summary_detailed.csv', index=False)

# Save to Excel for sharing
summary_df.to_excel('probe_summary_detailed.xlsx', index=False)

# Optional: Display the summary
print(summary_df)


In [ ]:
import pandas as pd

# Function to get probe data from manifests
def get_probes_from_manifest(manifest_name):
    manifest = Manifest(manifest_name)
    return set(manifest.data_frame['IlmnID'])

# Extract probe IDs from EPICv2 as a set for faster lookups
epicv2_probes_set = get_probes_from_manifest("epicv2")

# Load GrimAge2 feature sets and extract unique probes as a set
df_grimage_subcomponents = pd.read_csv('grimage2_subcomponents.csv', index_col=0)
grimage2_probes_set = set([probe for probe in df_grimage_subcomponents['var'].unique() if probe.startswith('cg')])

# Filter probe names that start with 'cg' from the methylation data
cg_probes_set = set([col for col in methylation_data_final_pd_prepared.columns if col.startswith('cg')])

# Calculate specific missing probes
missing_grimage2_in_epicv2 = grimage2_probes_set - epicv2_probes_set
missing_grimage2_in_methylation = grimage2_probes_set - cg_probes_set

# Probes missing in methylation but not in EPICv2
missing_in_methylation_only = missing_grimage2_in_methylation - missing_grimage2_in_epicv2

# Create a detailed summary DataFrame with the necessary missing probes
missing_specific_summary_df = pd.DataFrame({
    'Description': [
        'GrimAge2 probes missing in EPICv2',
        'GrimAge2 probes missing in methylation data',
        'GrimAge2 probes missing in only methylation but not EPICv2'
    ],
    'Number of Missing Probes': [
        len(missing_grimage2_in_epicv2),
        len(missing_grimage2_in_methylation),
        len(missing_in_methylation_only)
    ],
    'Missing Probes': [
        ', '.join(list(missing_grimage2_in_epicv2)),
        ', '.join(list(missing_grimage2_in_methylation)),
        ', '.join(list(missing_in_methylation_only))
    ]
})

# Sort the DataFrame by 'Number of Missing Probes' in descending order
missing_specific_summary_df = missing_specific_summary_df.sort_values(by='Number of Missing Probes', ascending=False)

# Save to CSV for sharing
missing_specific_summary_df.to_csv('detailed_missing_probes_summary.csv', index=False)

# Save to Excel for sharing
missing_specific_summary_df.to_excel('detailed_missing_probes_summary.xlsx', index=False)

# Optional: Display the summary
print(missing_specific_summary_df)


In [ ]:
# Convert DataFrames to AnnData objects
adata_final = pya.pp.df_to_adata(methylation_data_final_pd_prepared, metadata_cols=['Female','Age','Diagnosis'], imputer_strategy='knn')
adata_subset = pya.pp.df_to_adata(methylation_data_subset_pd_prepared, metadata_cols=['Female','Age','Diagnosis'], imputer_strategy='knn')

In [ ]:
# adata_subset.uns
#methylation_data_subset_pd_prepared.columns

In [ ]:
logger = pya.logger.LoggerManager.gen_logger("example")
metadata = pya.utils._utils.load_clock_metadata("pyaging_data", logger)
all_clocks = list(metadata.keys())
print("All available clocks:", all_clocks)
#all_clocks = pya.utils._utils.show_all_clocks()

In [ ]:
# Predict age using GrimAge2 for both datasets
# Iterate over each clock and attempt to predict age
for clock in all_clocks:
    try:
        print(f"Processing clock: {clock}")
        pya.pred.predict_age(adata_final, clock_names=[clock], dir="pyaging_data", batch_size=1024, clean=True, verbose=True)
        print(f"Successfully processed clock: {clock}")
    except Exception as e:
        print(f"Failed to process clock: {clock} due to error: {e}")
#pya.pred.predict_age(adata_subset)

In [ ]:
# Inspect the columns in the obs DataFrame
#print(adata_subset.obs.columns)
print(adata_final.obs.columns)

In [ ]:
# View the first few rows of predictions
# print(adata_final.obs[['GrimAge2']].head())
#print(adata_subset.obs.head())
print(adata_final.obs.head())

# Save predictions to CSV files
# adata_final.obs[['GrimAge2']].to_csv('GrimAge2_predictions_final.csv')
#adata_subset.obs.to_csv('GrimAge2_predictions_subset.csv')
adata_final.obs.to_csv('GrimAge2_predictions_final.csv')

In [ ]:
# Extract clock prediction columns (starting from the 4th column onwards)
clock_columns = adata_final.obs.iloc[:, 3:]

# Calculate correlations with the 'Age' column for different methods
pearson_corr = clock_columns.corrwith(adata_final.obs['Age'], method='pearson')
spearman_corr = clock_columns.corrwith(adata_final.obs['Age'], method='spearman')
kendall_corr = clock_columns.corrwith(adata_final.obs['Age'], method='kendall')

# Combine correlations into a single DataFrame
correlations = pd.DataFrame({
    'Pearson': pearson_corr,
    'Spearman': spearman_corr,
    'Kendall': kendall_corr
})

# Rank the clocks by their correlation with chronological age
correlations['Rank_Pearson'] = correlations['Pearson'].rank(ascending=False)
correlations['Rank_Spearman'] = correlations['Spearman'].rank(ascending=False)
correlations['Rank_Kendall'] = correlations['Kendall'].rank(ascending=False)

# Sort by Pearson correlation as an example
correlations_sorted = correlations.sort_values(by='Pearson', ascending=False)

# Display the ranked correlations
print(correlations_sorted.head())

In [ ]:
# Extract relevant information from adata_final.uns
missing_data = {}
for key in adata_final.uns.keys():
    if 'percent_na' in key or 'missing_features' in key:
        missing_data[key] = adata_final.uns[key]

# Convert the dictionary into a DataFrame for easier manipulation
missing_df = pd.DataFrame(list(missing_data.items()), columns=['Clock', 'Value'])

# Separate the Percent NA and Missing Features
percent_na_df = missing_df[missing_df['Clock'].str.contains('percent_na')].copy()
missing_features_df = missing_df[missing_df['Clock'].str.contains('missing_features')].copy()

# Rank by Percent NA using .loc to avoid SettingWithCopyWarning
percent_na_df.loc[:, 'Rank_Percent_NA'] = percent_na_df['Value'].rank(ascending=False)

# Rank by Missing Features using .loc to avoid SettingWithCopyWarning
missing_features_df.loc[:, 'Rank_Missing_Features'] = missing_features_df['Value'].rank(ascending=False)

# Combine the rankings into a single DataFrame
combined_rankings = pd.merge(percent_na_df, missing_features_df, on='Clock', how='outer')

# Sort by the ranks
combined_rankings_sorted = combined_rankings.sort_values(by=['Rank_Percent_NA', 'Rank_Missing_Features'], ascending=False)

# Display the sorted DataFrame
print(combined_rankings_sorted)

In [ ]:
adata_final.uns

In [ ]:
# Replace inf with NaN and drop rows where either value is NaN
valid_data = adata_subset.obs[['Age', 'grimage2']].replace([np.inf, -np.inf], np.nan).dropna()

# Extract the synchronized age and grimage2 arrays
age_clean = valid_data['Age']
grimage2_clean = valid_data['grimage2']

# Calculate Pearson correlation
pearson_corr, pearson_p_value = stats.pearsonr(age_clean, grimage2_clean)

# Calculate Spearman correlation (robust to non-linearity)
spearman_corr, spearman_p_value = stats.spearmanr(age_clean, grimage2_clean)

# Calculate Kendall correlation (robust to outliers)
kendall_corr, kendall_p_value = stats.kendalltau(age_clean, grimage2_clean)

# Display the correlation results
# Define a threshold for extremely small p-values
threshold = 1e-320

# Display the correlation results with adjusted p-values
print(f"Pearson correlation: {pearson_corr:.3f}, p-value: {'< {:.0e}'.format(threshold) if pearson_p_value < threshold else '{:.3e}'.format(pearson_p_value)}")
print(f"Spearman correlation: {spearman_corr:.3f}, p-value: {'< {:.0e}'.format(threshold) if spearman_p_value < threshold else '{:.3e}'.format(spearman_p_value)}")
print(f"Kendall correlation: {kendall_corr:.3f}, p-value: {'< {:.0e}'.format(threshold) if kendall_p_value < threshold else '{:.3e}'.format(kendall_p_value)}")

In [ ]:
# Set the theme and style for the plot
sns.set_theme(style="whitegrid")

# Create the figure and axis objects
plt.figure(figsize=(14, 10))

# Define a modern color palette with strong contrast and color-blind safety
colors = ['#0077BB', '#EE3377', '#EE7733']  # Vivid Blue, Vibrant Pink, Bright Orange

# Use seaborn's regplot for scatter plot with regression line
sns.regplot(
    x=age_clean,
    y=grimage2_clean,
    ci=95,
    scatter_kws={"s": 40, "alpha": 0.2, "color": colors[0]},  # Vivid Blue points with adjusted opacity
    line_kws={"color": colors[1], "linewidth": 3, "alpha": 1},  # Vibrant Pink regression line, thicker and more opaque
)

# Add a line plot with confidence interval
sns.lineplot(
    x=age_clean,
    y=grimage2_clean,
    errorbar=('ci', 99),
    lw=2.5,  # Increase line width for better visibility
    color=colors[2],  # Bright Orange line plot
    alpha=0.5,  # Increase opacity for confidence intervals
)

# Add titles and labels with enhanced font size
plt.title('Correlation between Age and GrimAge2', fontsize=24, weight='bold', pad=20)
plt.xlabel('Age (Yrs)', fontsize=18, weight='bold')
plt.ylabel('GrimAge2 (Yrs)', fontsize=18, weight='bold')

# Define a threshold for extremely small p-values
threshold = 1e-320

# Create a legend with correlation statistics
legend_text = (
    f"Pearson r = {pearson_corr:.3f}, p = {'< {:.0e}'.format(threshold) if pearson_p_value < threshold else '{:.3e}'.format(pearson_p_value)}\n"
    f"Spearman ρ = {spearman_corr:.3f}, p = {'< {:.0e}'.format(threshold) if spearman_p_value < threshold else '{:.3e}'.format(spearman_p_value)}\n"
    f"Kendall τ = {kendall_corr:.3f}, p = {'< {:.0e}'.format(threshold) if kendall_p_value < threshold else '{:.3e}'.format(kendall_p_value)}"
)
plt.text(0.05, 0.95, legend_text, ha='left', va='top', transform=plt.gca().transAxes, fontsize=16,
         bbox=dict(boxstyle="round,pad=0.4", edgecolor="gray", facecolor="white", alpha=0.8))

# Customize the ticks and labels for better readability
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Add grid lines
plt.grid(visible=True, color='gray', linestyle='--', linewidth=0.5, alpha=0.3)

# Remove the top and right spines for a cleaner look
sns.despine()

# Adjust the plot margins
plt.tight_layout()

# Save the plot as an SVG file for high resolution
plt.savefig('Age_vs_GrimAge2_Correlation_Accessible.svg', format='svg', dpi=4000, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
# Calculate the number and percentage of missing probes
total_probes = len(grimage2_probes)
missing_probes_count = len(grimage2_missing_features)
missing_probes_percent = (missing_probes_count / total_probes) * 100

# Set the theme and style for the plot
sns.set_theme(style="whitegrid")

# Define a modern color palette with strong contrast and color-blind safety
colors = ['#0077BB', '#EE3377', '#EE7733']  # Vivid Blue, Vibrant Pink, Bright Orange

# Create a figure with two subplots, giving more space to the main plot
fig, ax = plt.subplots(1, 2, figsize=(30, 12), gridspec_kw={'width_ratios': [2, 1]})

# First subplot: Correlation between Age and GrimAge2
sns.regplot(
    x=age_clean,
    y=grimage2_clean,
    ci=95,
    scatter_kws={"s": 40, "alpha": 0.2, "color": colors[0]},  # Vivid Blue points with adjusted opacity
    line_kws={"color": colors[1], "linewidth": 3, "alpha": 1},  # Vibrant Pink regression line, thicker and more opaque
    ax=ax[0]
)
sns.lineplot(
    x=age_clean,
    y=grimage2_clean,
    errorbar=('ci', 99),
    lw=2.5,  # Increase line width for better visibility
    color=colors[2],  # Bright Orange line plot
    alpha=0.5,  # Increase opacity for confidence intervals
    ax=ax[0]
)
ax[0].set_title('Correlation Between Age and GrimAge2', fontsize=28, weight='bold', pad=20)
ax[0].set_xlabel('Age (Yrs)', fontsize=22, weight='bold')
ax[0].set_ylabel('GrimAge2 (Yrs)', fontsize=22, weight='bold')

# Add legend with correlation statistics
ax[0].text(0.05, 0.95, legend_text, ha='left', va='top', transform=ax[0].transAxes, fontsize=20,
           bbox=dict(boxstyle="round,pad=0.4", edgecolor="gray", facecolor="white", alpha=0.8))

# Customize ticks and grid
ax[0].tick_params(axis='both', labelsize=18)
ax[0].grid(visible=True, color='gray', linestyle='--', linewidth=0.5, alpha=0.3)
sns.despine(ax=ax[0])

# Second subplot: Missing GrimAge2 probes (Pie chart)
labels = ['Present', 'Missing']
sizes = [total_probes - missing_probes_count, missing_probes_count]
colors = [colors[0], colors[1]]
explode = (0.05, 0.05)  # Slightly explode the slices for better visibility

ax[1].pie(
    sizes,
    labels=labels,
    autopct=lambda p: f'{p:.1f}%\n({int(p * total_probes / 100):,})',
    colors=colors,
    startangle=140,
    explode=explode,
    textprops={'fontsize': 20, 'weight': 'bold'}
)
ax[1].set_title('GrimAge2 Probes in Methylation Data', fontsize=28, weight='bold', pad=20)

# Adjust the plot margins
plt.tight_layout()

# Save the plot as an SVG file for high resolution
plt.savefig('Age_vs_GrimAge2_Correlation_MissingProbes_Pie_Updated.svg', format='svg', dpi=4000, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
# import { ScatterChart, Scatter, XAxis, YAxis, ZAxis, Tooltip, Legend, ResponsiveContainer } from 'recharts';

# const generateMockData = (n) => {
#   const data = [];
#   for (let i = 0; i < n; i++) {
#     const age = Math.floor(Math.random() * 60) + 20;
#     const grimAge = age + (Math.random() * 20 - 10);
#     const isBipolar = Math.random() > 0.5;
#     data.push({
#       age: age,
#       grimAge: grimAge,
#       group: isBipolar ? 'Bipolar' : 'Control',
#     });
#   }
#   return data;
# };

# const data = generateMockData(100);

# const AgeCorrelationPlot = () => {
#   return (
#     <ResponsiveContainer width="100%" height={400}>
#       <ScatterChart margin={{ top: 20, right: 20, bottom: 20, left: 20 }}>
#         <XAxis type="number" dataKey="age" name="Chronological Age" unit="years" />
#         <YAxis type="number" dataKey="grimAge" name="Biological Age (GrimAge)" unit="years" />
#         <ZAxis type="category" dataKey="group" name="Group" />
#         <Tooltip cursor={{ strokeDasharray: '3 3' }} />
#         <Legend />
#         <Scatter name="Bipolar" data={data.filter(d => d.group === 'Bipolar')} fill="#8884d8" />
#         <Scatter name="Control" data={data.filter(d => d.group === 'Control')} fill="#82ca9d" />
#       </ScatterChart>
#     </ResponsiveContainer>
#   );
# };

# export default AgeCorrelationPlot;

In [ ]:
# Summarize original diagnosis counts
diagnosis_counts = adata_subset.obs['Diagnosis'].value_counts()
print(diagnosis_counts)

# Regroup all Diagnosis entries into Bipolar diagnosis vs Other
df = adata_subset.obs.copy()
df['Diagnosis'] = df['Diagnosis'].apply(lambda x: 'Bipolar' if 'Bipolar' in x else 'Other')
diagnosis_counts_new = df['Diagnosis'].value_counts()
print(diagnosis_counts_new)
print(df)

In [ ]:
# Replace inf with NaN and drop rows where either value is NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(subset=['Age', 'grimage2'], inplace=True)

# Calculate the difference between GrimAge2 and Age
df['Difference'] = df['grimage2'] - df['Age']

# Standardize the difference for clustering
scaler = StandardScaler()
df['Scaled_Difference'] = scaler.fit_transform(df[['Difference']])

# Perform K-means clustering, experimenting with different cluster numbers
n_clusters = 2  # Two clusters: Aligned and Accelerated Aging
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['Cluster'] = kmeans.fit_predict(df[['Scaled_Difference']])

# Map the clusters to meaningful names
df['Group'] = df['Cluster'].replace({0: 'Aligned', 1: 'Accelerated Aging'})

# Replace 'Female' column values with 'Female' and 'Male' labels
df['Sex'] = df['Female'].replace({1.0: 'Female', 0.0: 'Male'})

# Create a new column that combines Group and Sex for coloring
df['Group_Sex'] = df['Group'] + ' - ' + df['Sex']

# Use a colorblind-safe palette with higher contrast
color_palette = {
    'Aligned - Female': '#0072B2',       # Bright blue
    'Aligned - Male': '#D55E00',         # Bright orange
    'Accelerated Aging - Female': '#009E73',  # Bright green
    'Accelerated Aging - Male': '#F0E442'     # Bright yellow
}

# Plot the Groups and indicate sex
plt.figure(figsize=(14, 10))
sns.set_style("whitegrid")
scatter = sns.scatterplot(
    data=df,
    x='Age',
    y='grimage2',
    hue='Group_Sex',
    style='Group_Sex',  # Indicating sex by marker style
    markers={'Aligned - Female': 'o', 'Aligned - Male': 's', 'Accelerated Aging - Female': '^', 'Accelerated Aging - Male': 'X'},
    palette=color_palette,
    s=120,
    alpha=0.8
)

# Define the concept of "Alignment" in the title
alignment_text = "Alignment = Grimage2 - Age (Aligned: within ±5 years or Decelerated, Accelerated: > 10 years)"
plt.title(f'Age vs GrimAge2: Clustered by Biological Age Alignment and Indicated by Sex\n{alignment_text}', fontsize=18, fontweight='bold')

# Customize the plot
plt.xlabel('Age (Yrs)', fontsize=18)
plt.ylabel('GrimAge2 (Yrs)', fontsize=18)
plt.legend(title='Group & Sex', fontsize=12, title_fontsize=14)
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()

# Save and show the plot
plt.savefig('Age_vs_GrimAge2_Biological_Age_Alignment_Sex_Clustering_Optimized.svg', format='svg', dpi=4000, bbox_inches='tight')
plt.show()

# Summary of group characteristics
group_summary = df.groupby(['Group', 'Sex']).agg({
    'Age': ['mean', 'std'],
    'grimage2': ['mean', 'std'],
    'Difference': ['mean', 'std'],
    'Group': 'size'
}).rename(columns={'Group': 'Count'}).reset_index()

print(group_summary)

In [ ]:
# Define the subplots
fig, axes = plt.subplots(1, 2, figsize=(24, 10))

# Plot 1: Original plot remains unchanged
sns.scatterplot(
    data=df,
    x='Age',
    y='grimage2',
    hue='Group_Sex',
    style='Group_Sex',
    markers={'Aligned - Female': 'o', 'Aligned - Male': 's', 'Accelerated Aging - Female': '^', 'Accelerated Aging - Male': 'X'},
    palette=color_palette,
    s=120,
    alpha=0.8,
    ax=axes[0]
)
alignment_text = "Alignment = Grimage2 - Age (Aligned: within ±5 years or Decelerated, Accelerated: > 10 years)"
axes[0].set_title(f'Age vs GrimAge2: Clustered by Biological Age Alignment and Indicated by Sex\n{alignment_text}', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Age (Yrs)', fontsize=14)
axes[0].set_ylabel('GrimAge2 (Yrs)', fontsize=14)
axes[0].grid(True, linestyle='--', alpha=0.3)

# Plot 2: Enhanced visualization (Age vs GrimAge2, colored by Bipolar vs Other)
sns.scatterplot(
    data=df_bipolar_other,
    x='Age',
    y='grimage2',
    hue='Diagnosis',
    style='Sex',
    markers={'Female': 'o', 'Male': 's'},
    palette={'Bipolar': '#1f77b4', 'Other': '#ff7f0e'},  # More distinct colors
    s=150,  # Increased marker size
    alpha=0.7,
    ax=axes[1]
)

# Add regression lines to Plot 2
sns.regplot(
    data=df_bipolar_other[df_bipolar_other['Diagnosis'] == 'Bipolar'],
    x='Age',
    y='grimage2',
    scatter=False,
    color='#1f77b4',
    ax=axes[1],
    label='Bipolar Fit'
)
sns.regplot(
    data=df_bipolar_other[df_bipolar_other['Diagnosis'] == 'Other'],
    x='Age',
    y='grimage2',
    scatter=False,
    color='#ff7f0e',
    ax=axes[1],
    label='Other Fit'
)

axes[1].set_title(f'Age vs GrimAge2: Colored by Bipolar vs Other', fontsize=16, fontweight='bold')
axes[1].set_xlabel('Age (Yrs)', fontsize=14)
axes[1].set_ylabel('GrimAge2 (Yrs)', fontsize=14)
axes[1].grid(True, linestyle='--', alpha=0.3)
axes[1].legend(title='Diagnosis')

# Perform ANOVA to check for significant differences between Bipolar and Other
model = ols('grimage2 ~ Age * Diagnosis', data=df_bipolar_other).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# Display ANOVA results on the plot
textstr = f'ANOVA p-value: {anova_table["PR(>F)"]["Age:Diagnosis"]:.3e}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
axes[1].text(0.05, 0.95, textstr, transform=axes[1].transAxes, fontsize=14,
             verticalalignment='top', bbox=props)

# Adjust layout and show both plots
plt.tight_layout()

# Save the combined plot
plt.savefig('Enhanced_Age_vs_GrimAge2_Bipolar_vs_Other_with_Stats.svg', format='svg', dpi=4000, bbox_inches='tight')
plt.show()

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns

# Define the subplots
fig, axes = plt.subplots(2, 2, figsize=(20, 15))

# Facet Grid for Bipolar and Other
sns.scatterplot(
    data=df_bipolar_other[df_bipolar_other['Diagnosis'] == 'Bipolar'],
    x='Age',
    y='grimage2',
    hue='Sex',
    palette={'Female': '#1f77b4', 'Male': '#ff7f0e'},
    s=100,
    alpha=0.7,
    ax=axes[0, 0]
)
sns.regplot(
    data=df_bipolar_other[df_bipolar_other['Diagnosis'] == 'Bipolar'],
    x='Age',
    y='grimage2',
    scatter=False,
    color='#1f77b4',
    ax=axes[0, 0]
)
axes[0, 0].set_title('Bipolar: Age vs GrimAge2', fontsize=16)
axes[0, 0].grid(True, linestyle='--', alpha=0.3)

sns.scatterplot(
    data=df_bipolar_other[df_bipolar_other['Diagnosis'] == 'Other'],
    x='Age',
    y='grimage2',
    hue='Sex',
    palette={'Female': '#1f77b4', 'Male': '#ff7f0e'},
    s=100,
    alpha=0.7,
    ax=axes[0, 1]
)
sns.regplot(
    data=df_bipolar_other[df_bipolar_other['Diagnosis'] == 'Other'],
    x='Age',
    y='grimage2',
    scatter=False,
    color='#ff7f0e',
    ax=axes[0, 1]
)
axes[0, 1].set_title('Other: Age vs GrimAge2', fontsize=16)
axes[0, 1].grid(True, linestyle='--', alpha=0.3)

# Residuals plot for Bipolar and Other
sns.residplot(
    data=df_bipolar_other,
    x='Age',
    y='grimage2',
    lowess=True,
    color='blue',
    ax=axes[1, 0]
)
axes[1, 0].set_title('Residuals: Age vs GrimAge2 (Bipolar)', fontsize=16)
axes[1, 0].grid(True, linestyle='--', alpha=0.3)

sns.residplot(
    data=df_bipolar_other,
    x='Age',
    y='grimage2',
    lowess=True,
    color='orange',
    ax=axes[1, 1]
)
axes[1, 1].set_title('Residuals: Age vs GrimAge2 (Other)', fontsize=16)
axes[1, 1].grid(True, linestyle='--', alpha=0.3)

# Adjust layout
plt.tight_layout()

# Save the plot
plt.savefig('Facet_Age_vs_GrimAge2_Bipolar_vs_Other_with_Residuals.svg', format='svg', dpi=4000, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
import statsmodels.api as sm

# Assume df already exists with the necessary modifications

# Separate the data for Bipolar and Other
df_bipolar = df[df['Diagnosis'] == 'Bipolar']
df_other = df[df['Diagnosis'] == 'Other']

# Define the subplots
fig, axes = plt.subplots(1, 2, figsize=(24, 10), sharey=True)

# Bipolar Plot
sns.scatterplot(
    data=df_bipolar,
    x='Age',
    y='grimage2',
    hue='Sex',
    style='Sex',
    markers={'Female': 'o', 'Male': 's'},
    palette={'Female': '#1f77b4', 'Male': '#ff7f0e'},
    s=150,  # Increased marker size
    alpha=0.7,
    ax=axes[0]
)
axes[0].set_title('Bipolar: Age vs GrimAge2', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Age (Yrs)', fontsize=14)
axes[0].set_ylabel('GrimAge2 (Yrs)', fontsize=14)
axes[0].grid(True, linestyle='--', alpha=0.3)
sns.regplot(data=df_bipolar, x='Age', y='grimage2', scatter=False, color='darkblue', ax=axes[0])

# Other Plot
sns.scatterplot(
    data=df_other,
    x='Age',
    y='grimage2',
    hue='Sex',
    style='Sex',
    markers={'Female': '^', 'Male': 'X'},  # Different marker styles
    palette={'Female': '#2ca02c', 'Male': '#d62728'},
    s=150,
    alpha=0.7,
    ax=axes[1]
)
axes[1].set_title('Other: Age vs GrimAge2', fontsize=16, fontweight='bold')
axes[1].set_xlabel('Age (Yrs)', fontsize=14)
axes[1].set_ylabel('GrimAge2 (Yrs)', fontsize=14)
axes[1].grid(True, linestyle='--', alpha=0.3)
sns.regplot(data=df_other, x='Age', y='grimage2', scatter=False, color='darkgreen', ax=axes[1])

# Statistical Analysis: Mean, Median, and Std Deviation
stats_bipolar = df_bipolar[['Age', 'grimage2']].describe().loc[['mean', 'std']]
stats_other = df_other[['Age', 'grimage2']].describe().loc[['mean', 'std']]

# Display stats on plots
textstr_bipolar = f"Mean Age: {stats_bipolar.loc['mean', 'Age']:.2f} Yrs\nMean GrimAge2: {stats_bipolar.loc['mean', 'grimage2']:.2f} Yrs"
textstr_other = f"Mean Age: {stats_other.loc['mean', 'Age']:.2f} Yrs\nMean GrimAge2: {stats_other.loc['mean', 'grimage2']:.2f} Yrs"
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
axes[0].text(0.05, 0.95, textstr_bipolar, transform=axes[0].transAxes, fontsize=14, verticalalignment='top', bbox=props)
axes[1].text(0.05, 0.95, textstr_other, transform=axes[1].transAxes, fontsize=14, verticalalignment='top', bbox=props)

# Adjust layout and show both plots
plt.tight_layout()

# Save the combined plot
plt.savefig('Separate_Age_vs_GrimAge2_Bipolar_Other_with_Stats.svg', format='svg', dpi=4000, bbox_inches='tight')
plt.show()

In [ ]:
# Combine Age and Diagnosis to create an interaction term
df['Age_Diagnosis'] = df['Age'] * pd.Categorical(df['Diagnosis']).codes

# Fit a linear model with an interaction term between Age and Diagnosis
model = ols('grimage2 ~ Age * Diagnosis', data=df).fit()

# Perform ANCOVA to check for differences in slopes between the groups
anova_results = anova_lm(model)

# Plot model predictions with confidence intervals
sns.lmplot(
    data=df,
    x='Age',
    y='grimage2',
    hue='Diagnosis',
    col='Diagnosis',
    palette={'Bipolar': '#1f77b4', 'Other': '#ff7f0e'},
    height=7,
    aspect=1,
    markers=['o', 's'],
    ci=95
)

# Add annotations for statistical significance
plt.annotate(f'ANCOVA p-value: {anova_results["PR(>F)"].iloc[2]:.3e}', 
             xy=(0.5, 0.95), xycoords='figure fraction', 
             fontsize=12, color='red', ha='center')

# Customize plot appearance
plt.subplots_adjust(top=0.85)
plt.suptitle('Age vs GrimAge2: Analysis of Interaction by Diagnosis (Bipolar vs Other)', fontsize=16, fontweight='bold')

# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import statsmodels.api as sm

# Assuming df already exists with the necessary modifications

# Stratify by Sex
df_female = df[df['Sex'] == 'Female'].copy()
df_male = df[df['Sex'] == 'Male'].copy()

# Function to perform statistical tests and plot with LOWESS and Linear Regression
def plot_relationship(df, ax, title, color1, color2):
    # LOWESS Smoothing
    lowess_bipolar = lowess(df[df['Diagnosis'] == 'Bipolar']['grimage2'], df[df['Diagnosis'] == 'Bipolar']['Age'], frac=0.3)
    lowess_other = lowess(df[df['Diagnosis'] == 'Other']['grimage2'], df[df['Diagnosis'] == 'Other']['Age'], frac=0.3)
    
    # Plot LOWESS
    ax.plot(lowess_bipolar[:, 0], lowess_bipolar[:, 1], label='Bipolar LOWESS', color=color1, linestyle='-', linewidth=2)
    ax.plot(lowess_other[:, 0], lowess_other[:, 1], label='Other LOWESS', color=color2, linestyle='--', linewidth=2)
    
    # Linear Regression
    model_bipolar = LinearRegression().fit(df[df['Diagnosis'] == 'Bipolar']['Age'].values.reshape(-1, 1),
                                           df[df['Diagnosis'] == 'Bipolar']['grimage2'].values)
    model_other = LinearRegression().fit(df[df['Diagnosis'] == 'Other']['Age'].values.reshape(-1, 1),
                                         df[df['Diagnosis'] == 'Other']['grimage2'].values)
    
    # Plot Linear Regression
    ax.plot(df['Age'], model_bipolar.predict(df['Age'].values.reshape(-1, 1)), color=color1, linestyle=':', linewidth=1.5)
    ax.plot(df['Age'], model_other.predict(df['Age'].values.reshape(-1, 1)), color=color2, linestyle=':', linewidth=1.5)
    
    # Scatter Plot
    ax.scatter(df[df['Diagnosis'] == 'Bipolar']['Age'], df[df['Diagnosis'] == 'Bipolar']['grimage2'],
               color=color1, alpha=0.5)
    ax.scatter(df[df['Diagnosis'] == 'Other']['Age'], df[df['Diagnosis'] == 'Other']['grimage2'],
               color=color2, alpha=0.5)
    
    # Title and Labels
    ax.set_title(title, fontsize=16, fontweight='bold')
    ax.set_xlabel('Age', fontsize=14)
    ax.set_ylabel('GrimAge2', fontsize=14)
    ax.legend()
    
    # Perform ANOVA to compare the models
    df['interaction'] = df['Age'] * pd.Categorical(df['Diagnosis']).codes
    model = ols('grimage2 ~ Age * Diagnosis', data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    
    # Display p-value on plot
    p_value = anova_table["PR(>F)"]['Age:Diagnosis']
    ax.text(0.05, 0.95, f'p-value: {p_value:.3e}', transform=ax.transAxes,
            fontsize=12, color='red', verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))

# Plotting
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot for Females
plot_relationship(df_female, axes[0], 'Age vs GrimAge2 (Females)', color1='blue', color2='green')

# Plot for Males
plot_relationship(df_male, axes[1], 'Age vs GrimAge2 (Males)', color1='orange', color2='red')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from patsy import dmatrix
df = df.dropna(subset=['Age', 'grimage2', 'Diagnosis', 'Sex'])

# Stratify by Sex
df_female = df[df['Sex'] == 'Female'].copy()
df_male = df[df['Sex'] == 'Male'].copy()
from statsmodels.gam.api import GLMGam, BSplines

# Define the spline basis for Age
spline_basis = dmatrix("bs(Age, df=6)", {"Age": df['Age']}, return_type='dataframe')

# Fit GAM for Bipolar (Female)
X_female_bipolar = df_female[df_female['Diagnosis'] == 'Bipolar']['Age']
y_female_bipolar = df_female[df_female['Diagnosis'] == 'Bipolar']['grimage2']
gam_female_bipolar = GLMGam.from_formula("grimage2 ~ bs(Age, df=6)", data=df_female[df_female['Diagnosis'] == 'Bipolar'], smoother=BSplines(spline_basis, df=[6], degree=[3])).fit()

# Fit GAM for Other (Female)
X_female_other = df_female[df_female['Diagnosis'] == 'Other']['Age']
y_female_other = df_female[df_female['Diagnosis'] == 'Other']['grimage2']
gam_female_other = GLMGam.from_formula("grimage2 ~ bs(Age, df=6)", data=df_female[df_female['Diagnosis'] == 'Other'], smoother=BSplines(spline_basis, df=[6], degree=[3])).fit()

# Fit GAM for Bipolar (Male)
X_male_bipolar = df_male[df_male['Diagnosis'] == 'Bipolar']['Age']
y_male_bipolar = df_male[df_male['Diagnosis'] == 'Bipolar']['grimage2']
gam_male_bipolar = GLMGam.from_formula("grimage2 ~ bs(Age, df=6)", data=df_male[df_male['Diagnosis'] == 'Bipolar'], smoother=BSplines(spline_basis, df=[6], degree=[3])).fit()

# Fit GAM for Other (Male)
X_male_other = df_male[df_male['Diagnosis'] == 'Other']['Age']
y_male_other = df_male[df_male['Diagnosis'] == 'Other']['grimage2']
gam_male_other = GLMGam.from_formula("grimage2 ~ bs(Age, df=6)", data=df_male[df_male['Diagnosis'] == 'Other'], smoother=BSplines(spline_basis, df=[6], degree=[3])).fit()
print(gam_female_bipolar.summary())
print(gam_female_other.summary())
print(gam_male_bipolar.summary())
print(gam_male_other.summary())


In [ ]:
import statsmodels.api as sm
from patsy import dmatrix

# Example using BSplines from statsmodels
X_bipolar_female = dmatrix("bs(Age, df=6)", {"Age": df_female[df_female['Diagnosis'] == 'Bipolar']['Age']}, return_type='dataframe')
y_bipolar_female = df_female[df_female['Diagnosis'] == 'Bipolar']['grimage2']

gam_model = sm.GLM(y_bipolar_female, X_bipolar_female, family=sm.families.Gaussian())
gam_results = gam_model.fit()
print(gam_results.summary())

# Predict and plot
age_grid = np.linspace(df['Age'].min(), df['Age'].max(), 100)
X_pred = dmatrix("bs(age_grid, df=6)", {"age_grid": age_grid}, return_type='dataframe')
y_pred = gam_results.predict(X_pred)

plt.figure(figsize=(10, 6))
plt.plot(age_grid, y_pred, color='blue', label="Bipolar (Female)")
plt.scatter(df_female[df_female['Diagnosis'] == 'Bipolar']['Age'], y_bipolar_female, color='blue', alpha=0.5)
plt.xlabel('Age')
plt.ylabel('GrimAge2')
plt.legend()
plt.show()


In [ ]:
model = pya.models.GrimAge2()

In [ ]:
model.metadata.update({
    "clock_name": 'grimage2',
    "data_type": 'methylation',
    "species": 'Homo sapiens',
    "year": 2022,
    "approved_by_author": '⌛',
    "citation": "Lu, Ake T., et al. \"DNA methylation GrimAge version 2.\" Aging (Albany NY) 14.23 (2022): 9484.",
    "doi": "https://doi.org/10.18632/aging.204434",
    "research_only": True,
    "notes": None
})

In [ ]:
urls = [
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2_subcomponents.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/datMiniAnnotation3_Gold.csv",
]
dir = "."
logger = pya.logger.Logger()

for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

In [ ]:
# Load feature sets from CSV files
df = pd.read_csv('grimage2_subcomponents.csv', index_col=0)
df_grimage = pd.read_csv('grimage2.csv', index_col=0)

# Identify features
all_features = np.unique(df['var'])[2:].tolist() + ['Female', 'Age']
model.features = all_features

In [ ]:
# Function to load weights for subcomponents
def load_model_weights(y_pred, model_attr):
    indices = torch.tensor([all_features.index(item) for item in df.loc[df['Y.pred'] == y_pred, 'var'] if item in all_features]).long()
    model_layer = pya.models.LinearModel(input_dim=len(indices))
    model_layer.linear.weight.data = torch.tensor(df.loc[df['Y.pred'] == y_pred, 'beta'][1:].values).unsqueeze(0).float()
    model_layer.linear.bias.data = torch.tensor(df.loc[df['Y.pred'] == y_pred, 'beta'].iloc[0]).float()
    setattr(model, model_attr, model_layer)
    setattr(model, f'features_{model_attr}', indices)

# Apply the function to each subcomponent
components = {
    'DNAmPACKYRS': 'PACKYRS', 
    'DNAmadm': 'ADM', 
    'DNAmB2M': 'B2M',
    'DNAmCystatin_C': 'CystatinC', 
    'DNAmGDF_15': 'GDF15',
    'DNAmleptin': 'Leptin',
    'DNAmpai_1': 'PAI1',
    'DNAmTIMP_1': 'TIMP1',
    'DNAmlog.CRP': 'LogCRP',
    'DNAmlog.A1C': 'A1C'
}

for y_pred, model_attr in components.items():
    load_model_weights(y_pred, model_attr)

# Load base model weights
base_model = pya.models.LinearModel(input_dim=len(df_grimage))
base_model.linear.weight.data = torch.tensor(df_grimage['beta'].tolist()).unsqueeze(0).float()
base_model.linear.bias.data = torch.tensor([0]).float()
model.base_model = base_model

In [ ]:
reference_df = pd.read_csv('datMiniAnnotation3_Gold.csv', index_col=0)
model.reference_values = reference_df.loc[model.features[:-2]]['gold'].tolist() + [1, 65]  # Example: 65-year-old female

In [ ]:
model.preprocess_name = None
model.preprocess_dependencies = None
model.postprocess_name = 'cox_to_years'
model.postprocess_dependencies = None

In [ ]:
pya.utils.print_model_details(model)

In [ ]:
# Define the directory path
weights_dir = "../weights"

# Create the directory if it doesn't exist
os.makedirs(weights_dir, exist_ok=True)

# Save the model
torch.save(model, os.path.join(weights_dir, f"{model.metadata['clock_name']}.pt"))

In [ ]:
# print(methylation_data_subset_pd.columns)
missing_features = [feature for feature in model.features if feature not in methylation_data_subset_pd.columns]
# print("Missing features:", missing_features)
missing_features

In [ ]:
# Extract relevant features
# Adjust the model's features list to include only those present in the data
available_features = [feature for feature in model.features if feature in methylation_data_subset_pd.columns]
input_data_subset = methylation_data_subset_pd[available_features].values

# # Convert to tensor and run the model
# input_tensor_final = torch.tensor(input_data_final, dtype=torch.float32)
# model.eval()
# model.to(float)
# pred = model(input_tensor_final)
# print(pred)

# Convert to tensor and run the model
input_tensor_subset = torch.tensor(input_data_subset, dtype=torch.float32)
model.eval()
model.to(float)
pred = model(input_tensor_subset)
print(pred)

In [ ]:
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')        

In [ ]:
for item in os.listdir('.'):
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
    elif os.path.isdir(item):
        shutil.rmtree(item)